Tabu pretrazivanje

In [1]:
import random
import numpy as np
import chess
import chess.svg

In [2]:
class TabuSearch:
    def __init__(self, n, max_iterations, tabu_size):
        self.n = n  # broj kraljica
        self.max_iterations = max_iterations  # maksimalan broj iteracija
        self.tabu_size = tabu_size  # duzin tabu liste
        self.tabu_list = []
        self.solution = []
        self.min_conflicts = float('inf')
        
    def initial_solution(self):
        return [random.randint(0, self.n - 1) for _ in range(self.n)]
    
    def conflicts(self, state):
        conflicts = 0
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    conflicts += 1
        return conflicts
    
    def generate_neighbors(self, state):
        neighbors = []
        for col in range(self.n):
            for row in range(self.n):
                if state[col] != row:
                    neighbor = list(state)
                    neighbor[col] = row
                    neighbors.append(neighbor)
        return neighbors
    
    def tabu_search(self):
        current_solution = self.initial_solution()
        current_conflicts = self.conflicts(current_solution)
        iteration = 0
        
        while iteration < self.max_iterations and current_conflicts > 0:
            best_neighbor = None
            best_neighbor_conflicts = float('inf')
            
            # Generisanje susjeda i odabir najboljeg
            for neighbor in self.generate_neighbors(current_solution):
                neighbor_conflicts = self.conflicts(neighbor)
                if neighbor_conflicts < best_neighbor_conflicts and neighbor not in self.tabu_list:
                    best_neighbor = neighbor
                    best_neighbor_conflicts = neighbor_conflicts
            
            current_solution = best_neighbor
            current_conflicts = best_neighbor_conflicts
            
            # Update tabu liste
            self.tabu_list.append(best_neighbor)
            if len(self.tabu_list) > self.tabu_size:
                self.tabu_list.pop(0)
            
            iteration += 1
            
            if current_conflicts < self.min_conflicts:
                self.min_conflicts = current_conflicts
                self.solution = current_solution[:]
            
            print(f"Iteration {iteration}: conflicts = {current_conflicts}")
        
        print("Solution:", self.solution)
        print("Minimum conflicts:", self.min_conflicts)

In [3]:
n_queens = 8
max_iterations = 1000
tabu_size = 10
tabu_search = TabuSearch(n_queens, max_iterations, tabu_size)
tabu_search.tabu_search()

Iteration 1: conflicts = 5
Iteration 2: conflicts = 3
Iteration 3: conflicts = 2
Iteration 4: conflicts = 1
Iteration 5: conflicts = 1
Iteration 6: conflicts = 1
Iteration 7: conflicts = 1
Iteration 8: conflicts = 1
Iteration 9: conflicts = 0
Solution: [5, 2, 0, 6, 4, 7, 1, 3]
Minimum conflicts: 0


In [4]:
board = chess.Board()
board.clear_board()
i = 0
for s in tabu_search.solution:
    square = chess.square(i, s)
    i += 1
    board.set_piece_at(square, chess.Piece(chess.QUEEN, chess.WHITE))
    svg_board = chess.svg.board(board=board, size=640)
# Save image
with open('image.svg', 'w') as f:
    f.write(svg_board)